# <span style="color:gold">**Árbol de Decisión con información geológica**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este tutorial, revisarás un ejemplo del modelo de <span style="color:gold">árboles de decisión</span> en Geología.

Es necesario tener un conocimiento previo en geoquímica y petrología, también es necesario haber aprendido a usar [Pandas](1c_pandas.ipynb).

Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.


## **¿Qué es un árbol de decisión?**
***

De acuerdo con [IBM](https://www.ibm.com/es-es/topics/decision-trees#:~:text=Un%20%C3%A1rbol%20de%20decisi%C3%B3n%20es,nodos%20internos%20y%20nodos%20hoja.), un árbol de decisión es un <span style="color:gold">algoritmo de aprendizaje supervisado no paramétrico</span>, que se utiliza tanto para tareas de clasificación como de regresión. Tiene una estructura de árbol jerárquica, que consta de un nodo raíz, ramas, nodos internos y nodos hoja.

<img src="resources/decision_tree.png" alt="Árbol de decisión" width="700"/>

El aprendizaje del árbol de decisiones emplea una estrategia de **divide y vencerás** mediante la realización de una búsqueda codiciosa para identificar los puntos de división óptimos dentro de un árbol. Este proceso de división se repite de forma recursiva de arriba hacia abajo hasta que todos o la mayoría de los registros se hayan clasificado bajo etiquetas de clase específicas.

Que todos los puntos de datos se clasifiquen o no como conjuntos homogéneos depende en gran medida de la complejidad del árbol de decisión.

Los árboles más pequeños tienden a generar nodos hoja puros, es decir, puntos de datos en una sola clase.\
Sin embargo, a medida que un árbol crece en tamaño, se vuelve cada vez más difícil mantener esta pureza y, por lo general, da como resultado que haya muy pocos datos dentro de un subárbol determinado.\
Cuando esto ocurre, se conoce como fragmentación de datos y, a menudo, puede resultar en **sobreajustes**.



<img src="resources/decision_tree_geology.png" alt="Árbol de decisión geológico" width="700"/>

Como resultado, los árboles de decisión tienen preferencia por los árboles pequeños, lo cual es consistente con el principio de parsimonia en la Navaja de Occam. Es decir, "las entidades no deben multiplicarse más allá de la necesidad". Dicho de otra manera, los árboles de decisión deben agregar complejidad solo si es necesario, ya que la explicación más simple suele ser la mejor. Para reducir la complejidad y evitar el sobreajuste, generalmente se emplea la **poda (pruning)**.

La poda es un proceso que elimina las ramas que se dividen en características con poca importancia. Luego, el ajuste del modelo se puede evaluar mediante el proceso de validación cruzada.

### **¿Qué pasaría si utilizamos varios árboles de decisión para predecir un resultado?**

Otra forma en que los árboles de decisión pueden mantener su precisión es mediante la formación de un conjunto a través de un algoritmo de **bosque aleatorio (Random Forest)**. Este clasificador predice resultados más precisos, particularmente cuando los árboles individuales no están correlacionados entre sí.

## **Árboles de decisión en Python**

Empezaremos importando `pandas` para cargar el archivo `rocas.csv`.\
También importaremos algunas funciones de Sci-Kit Learn:
> **Sci-Kit Learn** es una librería utilizada para construir algoritmos de machine learning, la referenciamos dentro de Python como `sklearn`.

In [ ]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier      # Modelo de Árbol de Decisión
from sklearn.model_selection import train_test_split # Función para dividir los datos de entrenamiento y prueba
from sklearn.metrics import accuracy_score           # Función para medir la precisión del modelo

Cargamos el archivo `rocas.csv` ubicado en la carpeta `files`:

In [ ]:
data = pd.read_csv("files/rocas.csv")
data.head()

Ahora, seleccionamos las litologías de `Granodiorita` y `Andesita`:

In [ ]:
data = data[(data["Nombre"] == "Granodiorita") | (data["Nombre"] == "Andesita")].reset_index(drop=True)
data.head()

Crearemos una nueva columna llamada `Roca` y asignaremos un valor numérico que representará el tipo de litología: 0 para `Granodiorita` y 1 para `Andesita`:

In [ ]:
data["Roca"] = 0                                                   # Todos los valores en Roca = 0
data.loc[data[data["Nombre"] == "Andesita"].index, "Roca"] = 1     # Valores de Andesita en Roca = 1
data

Luego de hacer la transformación de datos, separamos las columnas de la siguiente forma:
- `X` : contiene la información numérica de concentraciones geoquímicas, usada para entrenar y probar el modelo.
- `y` : contiene la información de la columna `Roca`, la variable a predecir.

Usaremos el atributo `values` del DataFrame para convertir la información en arreglos de Numpy:

In [ ]:
X = data.iloc[:, 1:-1].values
y = data["Roca"].values

Una vez separado los datos, procedemos a separar la data de entrenamiento y de prueba usando la función `train_test_split`, el parámetro `test_size=0.10` representa la fracción de la data que será asignada al conjunto de prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

Ahora, crearemos el modelo y lo entrenaremos usando la data de entrenamiento:
> El parámetro `criterion="gini"` establece el criterio de división de los nodos y `max_depth=3` establece la profundidad del árbol de decisión.

In [ ]:
model = DecisionTreeClassifier(criterion="gini", max_depth=3)

### **¿Qué es la impureza de Gini?**
**CART** es una abreviatura de **árboles de clasificación y regresión (classification and regression trees)** y fue introducido por Leo Breiman.\
Este algoritmo generalmente utiliza la **impureza de Gini** para identificar el atributo ideal para la división.\
La impureza de Gini mide la frecuencia con la que se clasifica incorrectamente un atributo elegido al azar. Cuando se evalúa usando la impureza de Gini, un valor más bajo es más ideal. 

Procedemos a entrenar el modelo:

In [ ]:
model.fit(X_train, y_train)

Una vez entrenado el modelo, mediremos su exactitud usando la función `accuracy_score`:
> La **exactitud** representa la cantidad de predicciones que fueron correctas.\
> El parámetro `y_true` representa la data que se busca obtener y `y_pred` es la predicción realizada por el modelo.\
> Para predecir valores con el modelo, tenemos que usar el método `predict`.

In [ ]:
print(f"Accuracy Score - Entrenamiento: {accuracy_score(y_true=y_train, y_pred=model.predict(X_train)):.1%}")
print(f"Accuracy Score - Prueba: {accuracy_score(y_true=y_test, y_pred=model.predict(X_test)):.1%}")

### **¿Podemos visualizar un árbol de decisión?**
La respuesta es sí, para esto, utilizaremos las funciones `export_text` y `plot_tree` del módulo `sklearn.tree`:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import export_text, plot_tree      # Funciones para graficar el árbol de decisión

In [ ]:
data.head(1)

Crearemos una variable llamada `x_cols` para almacenar los nombres de las columnas de X:
> Seleccionamos las columnas a partir de la segunda posición (1) hasta antes del último (-1).

In [ ]:
# Columnas de la matriz X
x_cols = data.iloc[:, 1:-1].columns
print(x_cols)

Exportamos los parámetros del árbol de decisión usando la función `export_text`:

In [ ]:
text_representation = export_text(model)

Ahora, procedemos a graficar el árbol de decisión usando la función `plot_tree`:

In [ ]:
fig = plt.figure(figsize=(16, 10))

plot_tree(model, feature_names=x_cols, class_names=["Granodiorita", "Andesita"], filled=True)

plt.show()

Por último, observaremos la importancia de cada columna usando el atributo `feature_importances_`:

In [ ]:
print("Importancia de atributos")
for col, imp in zip(x_cols, model.feature_importances_):
    print(f"{col}: {imp:.1%}")

La **importancia de atributos** nos ayuda a determinar qué variables son las más importantes para entrenar el modelo.\
Observamos que la columna `SiO2` tiene una importancia muy alta comparada con el resto de columnas.\
Algunas columnas son irrelevantes para el entrenamiento del modelo.

## **Conclusiones**
- La naturaleza jerárquica de un árbol de decisión facilita ver qué atributos son los más importantes, por lo que son **más fáciles de interpretar** que otros modelos de aprendizaje automático.
- Los árboles de decisión tienen una serie de características que los hacen más flexibles que otros clasificadores.
- Los árboles de decisión se pueden aprovechar para tareas de clasificación y regresión, lo que los hace más flexibles que otros algoritmos.
- Los árboles de decisión complejos tienden a sobreajustarse y no se generalizan bien a los nuevos datos. Este escenario se puede evitar mediante el proceso de poda (pruning).
- Pequeñas variaciones dentro de los datos pueden producir un árbol de decisión muy diferente.

***